In [42]:
import numpy as np

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

import kernels

In [43]:
%load_ext autoreload
%autoreload 2
import solution
from solution import Model
from solution import cv_eval


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
#copy code to override default classes

In [45]:
#def main():
train_x_name = "train_x.csv"
train_y_name = "train_y.csv"

train_x = np.loadtxt(train_x_name, delimiter=',')
train_y = np.loadtxt(train_y_name, delimiter=',')

# load the test dateset
test_x_name = "test_x.csv"
test_x = np.loadtxt(test_x_name, delimiter=',')

In [39]:

np.shape(train_x)


(17250, 2)

 # Call Scikit-learn 

In [26]:
### TODO add a simple outer train, test split to select the hyperparameters (kernel type etc) in the inner CV loop 
kernel = kernels.sklearn_best()
K_cv = 10
model_config = {
    "kernel":kernel,
    "use_skit_learn":True,
    "model_preprocess_left_frac":1.0
}
cv_m = cv_eval(cv_splits = K_cv,
               cv_preprocess_left_frac = 0.05,
               model_config=model_config) # Choose whether to use custom model or scikit learn
val_cost_array = cv_m.run_cross_validation(train_x, train_y)

print(np.mean(val_cost_array))
print(np.std(val_cost_array))

(17250, 3)
i=0
logging_setup
2020-10-11 16:58:01,630 - root - INFO - Begin logging for single training run
2020-10-11 16:58:01,630 - root - INFO - Begin logging for single training run
2020-10-11 16:58:01,630 - root - INFO - Begin logging for single training run
training cost fn   0.053966
val cost fn        0.062423
val log likelihood fn        1379.772013


i=1
logging_setup
2020-10-11 16:58:03,756 - root - INFO - Begin logging for single training run
2020-10-11 16:58:03,756 - root - INFO - Begin logging for single training run
2020-10-11 16:58:03,756 - root - INFO - Begin logging for single training run
2020-10-11 16:58:03,756 - root - INFO - Begin logging for single training run
training cost fn   0.053888
val cost fn        0.049947
val log likelihood fn        1381.064066


i=2
logging_setup
2020-10-11 16:58:05,914 - root - INFO - Begin logging for single training run
2020-10-11 16:58:05,914 - root - INFO - Begin logging for single training run
2020-10-11 16:58:05,914 - root - IN

# Call FITC

In [46]:
### TODO add a simple outer train, test split to select the hyperparameters (kernel type etc) in the inner CV loop 
kernel = kernels.sklearn_best()
K_cv = 10
model_config = {
    "kernel":kernel,
    "use_skit_learn":False,
    "use_nystrom":False,
    "use_fitc":True,
    "model_preprocess_left_frac":1.0
}
cv_m = cv_eval(cv_splits = K_cv,
               cv_preprocess_left_frac = 0.05,
               model_config=model_config) # Choose whether to use custom model or scikit learn
val_cost_array = cv_m.run_cross_validation(train_x, train_y)

print(np.mean(val_cost_array))
print(np.std(val_cost_array))

(17250, 3)
i=0
logging_setup
2020-10-11 17:34:51,144 - root - INFO - Begin logging for single training run
2020-10-11 17:34:51,144 - root - INFO - Begin logging for single training run
2020-10-11 17:34:51,144 - root - INFO - Begin logging for single training run
2020-10-11 17:34:51,144 - root - INFO - Begin logging for single training run
2020-10-11 17:34:51,144 - root - INFO - Begin logging for single training run
2020-10-11 17:34:51,144 - root - INFO - Begin logging for single training run
2020-10-11 17:34:51,144 - root - INFO - Begin logging for single training run
2020-10-11 17:34:51,144 - root - INFO - Begin logging for single training run
2020-10-11 17:34:51,144 - root - INFO - Begin logging for single training run
2020-10-11 17:34:51,144 - root - INFO - Begin logging for single training run
2020-10-11 17:34:51,144 - root - INFO - Begin logging for single training run
2020-10-11 17:34:51,144 - root - INFO - Begin logging for single training run
2020-10-11 17:34:51,144 - root - IN

2020-10-11 17:34:51,764 - root - INFO - Begin logging for single training run
2020-10-11 17:34:51,764 - root - INFO - Begin logging for single training run
2020-10-11 17:34:51,764 - root - INFO - Begin logging for single training run
2020-10-11 17:34:51,783 - root - INFO - Using FITC
2020-10-11 17:34:51,783 - root - INFO - Using FITC
2020-10-11 17:34:51,783 - root - INFO - Using FITC
2020-10-11 17:34:51,783 - root - INFO - Using FITC
2020-10-11 17:34:51,783 - root - INFO - Using FITC
2020-10-11 17:34:51,783 - root - INFO - Using FITC
2020-10-11 17:34:51,783 - root - INFO - Using FITC
2020-10-11 17:34:51,783 - root - INFO - Using FITC
2020-10-11 17:34:51,783 - root - INFO - Using FITC
2020-10-11 17:34:51,783 - root - INFO - Using FITC
2020-10-11 17:34:51,783 - root - INFO - Using FITC
2020-10-11 17:34:51,783 - root - INFO - Using FITC
2020-10-11 17:34:51,783 - root - INFO - Using FITC
2020-10-11 17:34:51,783 - root - INFO - Using FITC
2020-10-11 17:34:51,783 - root - INFO - Using FITC
2

2020-10-11 17:34:52,437 - root - INFO - Using FITC
2020-10-11 17:34:52,437 - root - INFO - Using FITC
2020-10-11 17:34:52,921 - root - INFO - (990,)
2020-10-11 17:34:52,921 - root - INFO - (990,)
2020-10-11 17:34:52,921 - root - INFO - (990,)
2020-10-11 17:34:52,921 - root - INFO - (990,)
2020-10-11 17:34:52,921 - root - INFO - (990,)
2020-10-11 17:34:52,921 - root - INFO - (990,)
2020-10-11 17:34:52,921 - root - INFO - (990,)
2020-10-11 17:34:52,921 - root - INFO - (990,)
2020-10-11 17:34:52,921 - root - INFO - (990,)
2020-10-11 17:34:52,921 - root - INFO - (990,)
2020-10-11 17:34:52,921 - root - INFO - (990,)
2020-10-11 17:34:52,921 - root - INFO - (990,)
2020-10-11 17:34:52,921 - root - INFO - (990,)
2020-10-11 17:34:52,921 - root - INFO - (990,)
2020-10-11 17:34:52,921 - root - INFO - (990,)
2020-10-11 17:34:52,921 - root - INFO - (990,)
2020-10-11 17:34:52,921 - root - INFO - (990,)
2020-10-11 17:34:52,921 - root - INFO - (990,)
2020-10-11 17:34:52,921 - root - INFO - (990,)
2020-

2020-10-11 17:34:53,544 - root - INFO - (990,)
2020-10-11 17:34:53,544 - root - INFO - (990,)
2020-10-11 17:34:53,544 - root - INFO - (990,)
2020-10-11 17:34:53,544 - root - INFO - (990,)
training cost fn   0.088992
2020-10-11 17:34:53,569 - root - INFO - (15,)
2020-10-11 17:34:53,569 - root - INFO - (15,)
2020-10-11 17:34:53,569 - root - INFO - (15,)
2020-10-11 17:34:53,569 - root - INFO - (15,)
2020-10-11 17:34:53,569 - root - INFO - (15,)
2020-10-11 17:34:53,569 - root - INFO - (15,)
2020-10-11 17:34:53,569 - root - INFO - (15,)
2020-10-11 17:34:53,569 - root - INFO - (15,)
2020-10-11 17:34:53,569 - root - INFO - (15,)
2020-10-11 17:34:53,569 - root - INFO - (15,)
2020-10-11 17:34:53,569 - root - INFO - (15,)
2020-10-11 17:34:53,569 - root - INFO - (15,)
2020-10-11 17:34:53,569 - root - INFO - (15,)
2020-10-11 17:34:53,569 - root - INFO - (15,)
2020-10-11 17:34:53,569 - root - INFO - (15,)
2020-10-11 17:34:53,569 - root - INFO - (15,)
2020-10-11 17:34:53,569 - root - INFO - (15,)
20

2020-10-11 17:34:54,198 - root - INFO - (15,)
2020-10-11 17:34:54,198 - root - INFO - (15,)
2020-10-11 17:34:54,198 - root - INFO - (15,)
2020-10-11 17:34:54,198 - root - INFO - (15,)
2020-10-11 17:34:54,198 - root - INFO - (15,)
2020-10-11 17:34:54,198 - root - INFO - (15,)
2020-10-11 17:34:54,198 - root - INFO - (15,)
2020-10-11 17:34:54,198 - root - INFO - (15,)
2020-10-11 17:34:54,198 - root - INFO - (15,)
2020-10-11 17:34:54,198 - root - INFO - (15,)
2020-10-11 17:34:54,198 - root - INFO - (15,)
2020-10-11 17:34:54,198 - root - INFO - (15,)
val cost fn        0.119943
val log likelihood fn        -925.864330


i=5
logging_setup
2020-10-11 17:34:54,298 - root - INFO - Begin logging for single training run
2020-10-11 17:34:54,298 - root - INFO - Begin logging for single training run
2020-10-11 17:34:54,298 - root - INFO - Begin logging for single training run
2020-10-11 17:34:54,298 - root - INFO - Begin logging for single training run
2020-10-11 17:34:54,298 - root - INFO - Begin l

2020-10-11 17:34:54,946 - root - INFO - Begin logging for single training run
2020-10-11 17:34:54,946 - root - INFO - Begin logging for single training run
2020-10-11 17:34:54,946 - root - INFO - Begin logging for single training run
2020-10-11 17:34:54,946 - root - INFO - Begin logging for single training run
2020-10-11 17:34:54,946 - root - INFO - Begin logging for single training run
2020-10-11 17:34:54,946 - root - INFO - Begin logging for single training run
2020-10-11 17:34:54,946 - root - INFO - Begin logging for single training run
2020-10-11 17:34:54,946 - root - INFO - Begin logging for single training run
2020-10-11 17:34:54,946 - root - INFO - Begin logging for single training run
2020-10-11 17:34:54,946 - root - INFO - Begin logging for single training run
2020-10-11 17:34:54,946 - root - INFO - Begin logging for single training run
2020-10-11 17:34:54,946 - root - INFO - Begin logging for single training run
2020-10-11 17:34:54,946 - root - INFO - Begin logging for single

2020-10-11 17:34:55,587 - root - INFO - Begin logging for single training run
2020-10-11 17:34:55,587 - root - INFO - Begin logging for single training run
2020-10-11 17:34:55,587 - root - INFO - Begin logging for single training run
2020-10-11 17:34:55,587 - root - INFO - Begin logging for single training run
2020-10-11 17:34:55,587 - root - INFO - Begin logging for single training run
2020-10-11 17:34:55,587 - root - INFO - Begin logging for single training run
2020-10-11 17:34:55,587 - root - INFO - Begin logging for single training run
2020-10-11 17:34:55,587 - root - INFO - Begin logging for single training run
2020-10-11 17:34:55,587 - root - INFO - Begin logging for single training run
2020-10-11 17:34:55,587 - root - INFO - Begin logging for single training run
2020-10-11 17:34:55,587 - root - INFO - Begin logging for single training run
2020-10-11 17:34:55,587 - root - INFO - Begin logging for single training run
2020-10-11 17:34:55,587 - root - INFO - Begin logging for single

2020-10-11 17:34:56,263 - root - INFO - Begin logging for single training run
2020-10-11 17:34:56,263 - root - INFO - Begin logging for single training run
2020-10-11 17:34:56,263 - root - INFO - Begin logging for single training run
2020-10-11 17:34:56,263 - root - INFO - Begin logging for single training run
2020-10-11 17:34:56,263 - root - INFO - Begin logging for single training run
2020-10-11 17:34:56,263 - root - INFO - Begin logging for single training run
2020-10-11 17:34:56,263 - root - INFO - Begin logging for single training run
2020-10-11 17:34:56,263 - root - INFO - Begin logging for single training run
2020-10-11 17:34:56,263 - root - INFO - Begin logging for single training run
2020-10-11 17:34:56,263 - root - INFO - Begin logging for single training run
2020-10-11 17:34:56,263 - root - INFO - Begin logging for single training run
2020-10-11 17:34:56,263 - root - INFO - Begin logging for single training run
2020-10-11 17:34:56,263 - root - INFO - Begin logging for single

2020-10-11 17:34:56,898 - root - INFO - Begin logging for single training run
2020-10-11 17:34:56,898 - root - INFO - Begin logging for single training run
2020-10-11 17:34:56,898 - root - INFO - Begin logging for single training run
2020-10-11 17:34:56,898 - root - INFO - Begin logging for single training run
2020-10-11 17:34:56,898 - root - INFO - Begin logging for single training run
2020-10-11 17:34:56,898 - root - INFO - Begin logging for single training run
2020-10-11 17:34:56,898 - root - INFO - Begin logging for single training run
2020-10-11 17:34:56,898 - root - INFO - Begin logging for single training run
2020-10-11 17:34:56,898 - root - INFO - Begin logging for single training run
2020-10-11 17:34:56,898 - root - INFO - Begin logging for single training run
2020-10-11 17:34:56,898 - root - INFO - Begin logging for single training run
2020-10-11 17:34:56,898 - root - INFO - Begin logging for single training run
2020-10-11 17:34:56,898 - root - INFO - Begin logging for single

In [5]:
cv_m.models

[None, None, None, None, None, None, None, None, None, None]

# Call Nystrom

In [40]:
### TODO add a simple outer train, test split to select the hyperparameters (kernel type etc) in the inner CV loop 
kernel = kernels.sklearn_best()
K_cv = 10
model_config = {
    "kernel":kernel,
    "use_skit_learn":False,
    "use_nystrom":True,
    "use_fitc":False,
    "model_preprocess_left_frac":1.0
}
cv_m = cv_eval(cv_splits = K_cv,
               cv_preprocess_left_frac = 0.05,
               model_config=model_config) # Choose whether to use custom model or scikit learn
val_cost_array = cv_m.run_cross_validation(train_x, train_y)

print(np.mean(val_cost_array))
print(np.std(val_cost_array))

(17250, 3)
i=0
logging_setup
2020-10-11 17:34:02,524 - root - INFO - Begin logging for single training run
2020-10-11 17:34:02,524 - root - INFO - Begin logging for single training run
2020-10-11 17:34:02,524 - root - INFO - Begin logging for single training run
2020-10-11 17:34:02,524 - root - INFO - Begin logging for single training run
2020-10-11 17:34:02,524 - root - INFO - Begin logging for single training run
2020-10-11 17:34:02,524 - root - INFO - Begin logging for single training run
2020-10-11 17:34:02,524 - root - INFO - Begin logging for single training run
2020-10-11 17:34:02,524 - root - INFO - Begin logging for single training run
2020-10-11 17:34:02,524 - root - INFO - Begin logging for single training run
2020-10-11 17:34:02,524 - root - INFO - Begin logging for single training run
2020-10-11 17:34:02,524 - root - INFO - Begin logging for single training run
2020-10-11 17:34:02,524 - root - INFO - Begin logging for single training run
2020-10-11 17:34:02,524 - root - IN

/Users/laurieprelot/Documents/Projects/2020_probabilistic_AI/PAI_code/project1/solution.py:194: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  y = (np.random.multivariate_normal(means.ravel(), cov, 1)).flatten() #sample from the multivar normal


AttributeError: 'Model' object has no attribute 'K_x_x'

# Temporary 

In [ ]:
#check nystrom

In [ ]:
train_x_trunc = train_x[:100]
train_x_full = train_x[:10000]

In [ ]:
kernel = kernels.sklearn_best()
print(kernel(train_x_trunc,train_x_trunc))


In [ ]:
model_config = {"use_skit_learn":False,
                "use_nystrom":True,
                "nystrom_q":100,
                "kernel":kernel,
                "variance":0.01,
                "correct_y_pred":False,
                "preprocess_frac":0.05}
model = Model(model_config)

In [ ]:
model.fit_model(train_x,train_y)
model.


In [ ]:
Knn = kernels.rbf_kernel(train_x_full,train_x_full)
Kqq = kernels.rbf_kernel(train_x_trunc,train_x_trunc)
Knq = kernels.rbf_kernel(train_x_full,train_x_trunc)
res = np.linalg.eig(Kqq)
Lambdaqq = np.diag(res[0])
Uqq = res[1]

In [ ]:
#print(np.sum(res[0]).imag)
print(np.absolute(res[0]))
